In [1]:
## VISSIM Modules
import win32com.client as com
import os

## RL Modules
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("ERROR: GPU DEVICE NOT FOUND.")

from keras.models import load_model
    
## Data Management Modules
import pickle

## User Defined Modules
import Simulator_Functions as SF

from RLAgents import DQNAgent
from NParser import NetworkParser
from COMServer import COMServerDispatch, COMServerReload
from TupleToList import toList
## Other Modules
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

Default GPU Device: /device:GPU:0


Using TensorFlow backend.


In [2]:
## Network Model Parameters
Random_Seed = 42
model_name  = 'Single_Cross_Straight'
vissim_working_directory = 'C:\\Users\\acabrejasegea\\OneDrive - The Alan Turing Institute\\Desktop\\ATI\\0_TMF\\MLforFlowOptimisation\\Vissim\\'
program = 'DuelingDDQN' # DQN, DuelingDQN, DDQN, DuelingDDQN
reward_type = 'Queues'
state_type  = 'Queues'
PER = True
## Use of additional files?
flag_read_additionally  = False
## Load trained model?
Demo_Mode = False
load_trained = False
Quickmode = True
SaveResultsAgent = True
# Random demand
Random_Demand = False

In [3]:
## Data handling flags
# Flag for restarting the COM Server
reset_flag = True
#cache_flag = False
# If a fresh start is needed, all previous results from simulations are deleted
Start_Fresh = True
# Debug action
debug_action = False

In [ ]:
## RL Hyperparamenters
# Number of simulations, save every "n" episodes and copy weights with frequency "f"
episodes = 800
partial_save_at = 10
copy_weights_frequency = 5
reset_frequency = 10

# Timesteps per simulation (1 timestep = 0.1 sec), length for random population is a multiple of episode
simulation_length = 36000*1
memory_population_length = simulation_length*5

## State-Action Parameters
state_size = 4
action_size = 5

# Hyperparameters
batch_size = 128
memory_size = 10000
alpha   = 0.0001
gamma   = 0.95

# Exploration Schedule
epsilon_start = 1
epsilon_end   = 0.001
epsilon_decay = 1.2*(epsilon_end - epsilon_start)/(episodes-1)
#epsilon_decay = np.power(epsilon_end/epsilon_start, 1./(episodes+1)) # Geometric decay

# Demand Schedule
demands = [100, 200, 400, 600, 800, 1000]
# Session ID
Session_ID = 'Episodes'+str(episodes)+'_Program'+program

In [ ]:
if __name__ == "__main__":
    # Initialize storage
    reward_storage = []
    best_agent_weights = []
    loss=[]
    
    # If previous agents aren't to be loaded, proceed to memory population
    if not load_trained:
        # Initialize simulation
        if 'Vissim' not in globals() or Vissim == None:
            Vissim, Simulation, Network, cache_flag = COMServerDispatch(model_name, vissim_working_directory,\
                                                                        memory_population_length, Start_Fresh,\
                                                                        reset_flag = True, verbose = True)
        else:
            Vissim = com.Dispatch("Vissim.Vissim")
            Simulation, Network = COMServerReload(Vissim, model_name, vissim_working_directory,\
                                                  memory_population_length, Start_Fresh, reset_flag = True)
        
    # Setting Random Seed
    Vissim.Simulation.SetAttValue('RandSeed', Random_Seed)
    print ('Random seed set in simulator. Random Seed = '+str(Random_Seed))

    # Deploy Network Parser (crawl network)
    npa = NetworkParser(Vissim)
    print('NetworkParser has succesfully crawled the model network.')
    
    # Initialize agents
    if program == "DQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = False, Dueling = False) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DuelingDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = False, Dueling = True) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = True, Dueling = False) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DuelingDDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = True, Dueling = True) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    else:
        print("Incorrect Agent Class selected. Deployment could not be completed.")
        quit()
    
    if agents_deployed:
        print("Deployed {} agent(s) of the Class {}.".format(len(Agents), program))
    
    if Demo_Mode:
        Agents, loss = SF.load_agents(vissim_working_directory, model_name, Agents, Session_ID, loss, best = True)
        SF.run_simulation_episode(Agents, Vissim, state_type, state_size, memory_population_length, Demo_Mode)
        Vissim = None
    # Load previous trained data
    elif load_trained:
        Agents, loss = SF.load_agents(vissim_working_directory, model_name, Agents, Session_ID, loss, best = False)
    # If previous data isn't to be loaded, have an initial longer random run to populate memory
    else:
        print('Populating memory with Random Actions....')
        SF.Set_Quickmode(Vissim)
        SF.run_simulation_episode(Agents, Vissim, state_type, state_size, memory_population_length, Demo_Mode)
    
    # Iterations of the simulation
    for episode in range(episodes):
        # Completely re-dispatch server every N iterations for performance
        if episode % reset_frequency == 0 and episode !=0:
            Vissim = None
            Vissim, Simulation, Network, cache_flag = COMServerDispatch(model_name, vissim_working_directory,\
                                                                        simulation_length, Start_Fresh,\
                                                                        reset_flag = True, verbose = False)
            print("Redispatched")
        else:
            # If not the first episode, reset state at the start
            Simulation, Network = COMServerReload(Vissim, model_name, vissim_working_directory,\
                                                simulation_length, Start_Fresh, reset_flag = True)
        npa = NetworkParser(Vissim) 
        for index, agent in enumerate(Agents):
            agent.update_IDS(npa.signal_controllers_ids[index], npa)
            agent.episode_reward = []
        
        # Change demand for every episode
        if Random_Demand:
            for vehicle_input in range(1,5):
                Vissim.Net.VehicleInputs.ItemByKey(vehicle_input).SetAttValue('Volume(1)', demands[np.random.randint(0,len(demands)-1)])    
        
        # Run Episode at maximum speed

        SF.Set_Quickmode(Vissim)
        SF.run_simulation_episode(Agents, Vissim, state_type, state_size, simulation_length, Demo_Mode)
        
        # Calculate episode average reward
        reward_storage, average_reward = SF.average_reward(reward_storage, Agents, episode, episodes)
        best_agent_weights = SF.best_agent(reward_storage, average_reward, best_agent_weights, vissim_working_directory, model_name, Agents, Session_ID)
        
        # Train agent with experience of episode (indicated batch size)
        for agent in Agents:
            loss = agent.replay_batch(batch_size, episode, loss)
        # Security save for long trainings
        if SaveResultsAgent:
            if (episode+1)%partial_save_at == 0:
                SF.save_agents(vissim_working_directory, model_name, Agents, Session_ID, reward_storage, loss)
                print('Saved Partial results at the end of episode {}.'.format(episode+1))

    #Saving agents memory, weights and optimizer
    if SaveResultsAgent:
        SF.save_agents(vissim_working_directory, model_name, Agents, Session_ID, reward_storage, loss)
        print("Model, architecture, weights, optimizer, memory and training results succesfully saved. Succesfully Terminated.")
    
    # Plotting training progress
    plt.figure(figsize=(8,4.5))
    x_series = range(1,len(reward_storage)+1)
    fit = np.polyfit(x_series,reward_storage,1)
    fit_fn = np.poly1d(fit) 
    plt.plot(x_series,reward_storage, '-b', x_series, fit_fn(x_series), '--r')
    plt.xlabel('Episodes')
    plt.ylabel('Average ageng reward in episode')
    plt.title('Training evolution and trend')
    plt.gca().legend(('Episode Reward','Linear Trend'))
    plt.show()
    
    # Plotting training loss
    plt.figure(figsize=(8,4.5))
    x_series = range(1,len(loss)+1)
    plt.plot(x_series,loss, '-b')
    plt.xlabel('Training Epoch')
    plt.ylabel('Loss')
    plt.title('Model Loss')
    plt.gca().legend(('Loss'))
    plt.show()
    
    # Close Vissim
    Vissim = None

Working Directory set to: C:\Users\acabrejasegea\OneDrive - The Alan Turing Institute\Desktop\ATI\0_TMF\MLforFlowOptimisation\Vissim\
Generating Cache...
Cache generated.

****************************
*   COM Server dispatched  *
****************************

Loading Model File: Single_Cross_Straight.inpx ...
Load process successful
Simulation length set to 18000.0 seconds.
Results from Previous Simulations: Deleted. Fresh Start Available.
Fetched and containerized Simulation Object
Fetched and containerized Network Object 

*******************************************************
*                                                     *
*                 SETUP COMPLETE                      *
*                                                     *
*******************************************************

Random seed set in simulator. Random Seed = 42
NetworkParser has succesfully crawled the model network.
Instructions for updating:
Colocations handled automatically by placer.
Deploying in

Episode: 2/800, Epsilon:1.0, Average reward: -242.33
Prediction for [500,0,500,0] is: [[-222.40158   -291.4055       7.9351196    2.5771484 -363.84937  ]]
New best agent found. Saved in C:\Users\acabrejasegea\OneDrive - The Alan Turing Institute\Desktop\ATI\0_TMF\MLforFlowOptimisation\Vissim\Single_Cross_Straight\Single_Cross_Straight_Episodes800_ProgramDuelingDDQN_BestAgent0_Memory.p
[array([ 6.51121372, 22.0533308 ,  0.25562089, 15.27820096]), array([ 8.02053245, 17.27123601, 11.7728664 ,  6.25122048]), array([142.48380933,   0.        , 148.70406767,   0.58450791]), array([206.6365808 ,  10.06487251, 208.68266506,  16.38328842]), array([2.34140056e+02, 1.38459040e-01, 2.39257858e+02, 0.00000000e+00]), array([161.40952956,   6.07675412, 132.7839447 ,   2.3278031 ]), array([175.25258581,   6.20859941,  20.17160574,   1.45302714]), array([23.39262964,  3.49502336, 19.18418723,  0.        ]), array([ 61.79778163,   0.        , 144.27070385,   0.        ]), array([200.76542384,   0.     

Episode: 3/800, Epsilon:1.0, Average reward: -158.03
Prediction for [500,0,500,0] is: [[-230.27002    -297.92197       0.76327515   -8.450043   -371.0028    ]]
New best agent found. Saved in C:\Users\acabrejasegea\OneDrive - The Alan Turing Institute\Desktop\ATI\0_TMF\MLforFlowOptimisation\Vissim\Single_Cross_Straight\Single_Cross_Straight_Episodes800_ProgramDuelingDDQN_BestAgent0_Memory.p
[array([1.88375156e+02, 9.47540620e-02, 1.88433487e+02, 1.90327713e+00]), array([36.84780339,  0.        , 38.24392645,  0.        ]), array([197.75147343,   9.60004569,  94.25523202,  20.98565232]), array([ 94.2908584 ,   0.        , 198.46925285,   1.55228481]), array([234.73316521,   0.        , 234.57898793,   0.        ]), array([ 6.01060729,  3.47909838, 20.47050477,  4.68411038]), array([99.89402195,  1.55101195, 27.46897179,  0.9383114 ]), array([212.39636293,   0.51169096, 175.43447654,   0.        ]), array([148.48083364,   0.8920642 , 219.96882803,   0.        ]), array([171.42391736,  11.

Episode: 4/800, Epsilon:1.0, Average reward: -218.98
Prediction for [500,0,500,0] is: [[-237.26466   -304.93768     -6.3681793  -19.473267  -378.46967  ]]
[array([242.75712396,   0.        , 199.28069186,   0.54958683]), array([1.86117858e+02, 1.22390004e+00, 5.82000865e+01, 1.71405802e-01]), array([150.51748088,   0.        , 121.29260542,   0.31239186]), array([203.2087042 ,   6.30742603, 202.57672907,  11.05528639]), array([133.26252889,  12.80080143, 154.4366742 ,  15.16425546]), array([125.39097398,  10.87973753,   4.10290823,   4.40761474]), array([7.39205181, 3.81094229, 1.28240367, 8.17644416]), array([148.81076832,  11.16648182, 118.48821532,  19.57389885]), array([38.61215512,  0.39039731, 30.18529475,  0.        ]), array([130.44334088,   0.62176888,  48.57468823,   0.        ]), array([36.13383034,  0.        , 52.97602292,  0.        ]), array([129.12281052,   6.84127102,  55.96736942,   6.51063203]), array([ 78.04262538,   0.        , 227.38839895,   2.13549873]), array([

Episode: 5/800, Epsilon:0.99, Average reward: -187.53
Prediction for [500,0,500,0] is: [[-245.32483  -312.95615   -14.017502  -28.720383 -384.36163 ]]
[array([ 65.65123267,   4.67168419, 124.71568795,   6.86265088]), array([88.77822391, 10.98565314,  8.51500158,  6.19978418]), array([27.33150506,  8.74996017, 21.05085512,  5.69560762]), array([156.13186081,   6.8640114 ,  25.95639729,  21.8615472 ]), array([ 4.40458889,  5.22673662,  4.48388753, 18.5699448 ]), array([125.14379009,  29.63715912, 119.20141542,   4.32288229]), array([ 94.2908584 ,   0.        , 198.46925285,   1.55228481]), array([ 5.8433486 ,  4.55815944, 72.79266301,  3.11189992]), array([15.91444958,  3.48323836, 10.25194812,  3.12140987]), array([56.3426856 ,  1.88223378, 15.05361152,  7.76344451]), array([181.51472606,   0.        ,  26.0869345 ,   0.        ]), array([111.75325155,   1.29266103,  71.87413448,   1.56950543]), array([234.33524725,   0.79883789, 230.13485001,   1.4003892 ]), array([89.80993566,  1.2343

...

Episode: 29/800, Epsilon:0.96, Average reward: -160.36
Prediction for [500,0,500,0] is: [[-440.30508 -491.465   -227.88162 -255.81378 -532.60645]]
[array([89.80993566,  1.23437294, 48.42109635,  1.08119384]), array([236.89117068,   1.24369589, 235.96205956,   0.        ]), array([19.19081193,  1.14642244,  9.73004373,  2.18043966]), array([80.32107125, 16.68595577, 93.80187371, 12.66745954]), array([241.31040042,   0.        , 244.98577044,   0.        ]), array([203.78335595,   3.38773213, 236.77153569,   0.        ]), array([3.92303597, 1.59670796, 1.85826445, 4.03600224]), array([26.55470284,  4.16308196, 61.84181753, 10.82341463]), array([191.16236275,   2.13036462, 177.7092904 ,   3.06313575]), array([208.72617476,   0.        , 190.49617713,   0.        ]), array([158.84975715,  19.30426342, 166.23306   ,   0.61177863]), array([52.44448773,  0.27309931, 36.15072371,  7.40637879]), array([2.46251572e+02, 1.37465498e-01, 2.46054859e+02, 0.00000000e+00]), array([4.91651714e+01, 7.47

Episode: 30/800, Epsilon:0.96, Average reward: -188.48
Prediction for [500,0,500,0] is: [[-451.81433 -499.17386 -236.46997 -267.32187 -538.666  ]]
[array([ 3.58853016,  1.70302667,  1.86156366, 17.7658412 ]), array([25.26827397, 10.05975318, 15.5216719 ,  4.67340151]), array([178.80924126,   6.17170871, 167.17345442,   8.67744274]), array([2.46917475e+02, 2.15918653e-01, 2.40201824e+02, 0.00000000e+00]), array([69.77484315,  0.79097287, 82.37515153,  0.52971365]), array([89.80993566,  1.23437294, 48.42109635,  1.08119384]), array([13.58377775,  6.07985379, 93.5901908 ,  3.09571904]), array([198.15571941,   2.53145553, 203.33621324,   5.02507183]), array([177.53048936,  16.05158353, 175.81510434,   7.68414876]), array([194.99778628,   3.8853251 , 186.50569563,   3.88934816]), array([62.33744761,  2.16205644, 15.05623906,  5.2717229 ]), array([165.36596069,   5.40932862, 168.49661416,   2.8721687 ]), array([130.34744343,   0.25359061,  45.6272956 ,   1.27367327]), array([46.59171233, 13.

Sever Redispatched.
Redispatched
Episode: 31/800, Epsilon:0.95, Average reward: -254.47
Prediction for [500,0,500,0] is: [[-461.66534 -507.23578 -244.75864 -277.88385 -544.61957]]
[array([246.3816307 ,   0.        , 247.80223559,   0.        ]), array([183.88215866,  13.67460946,  92.73328736,   2.41843715]), array([167.90599964,  19.94077354, 122.49940367,   2.06504746]), array([61.63367738,  2.10034041, 30.19317304,  5.13570961]), array([216.47018189,   1.0050236 , 228.31390212,   0.60935115]), array([39.80275809,  0.5922528 , 20.43519326,  1.30628624]), array([197.4193528 ,   2.11355578, 109.39148292,   0.76652742]), array([198.80856254,   1.55569206,  84.35993996,   0.        ]), array([147.14722004,   0.53603879,  46.78291493,   0.        ]), array([ 1.51197999, 19.73644168,  5.82660452,  0.6837718 ]), array([89.80993566,  1.23437294, 48.42109635,  1.08119384]), array([ 6.98661775, 14.53745806, 13.09774911,  0.        ]), array([133.83504308,   6.37948677, 182.36138571,   0.      

Episode: 32/800, Epsilon:0.95, Average reward: -190.93
Prediction for [500,0,500,0] is: [[-470.958   -515.7163  -253.36047 -288.91864 -550.51697]]
[array([55.74779071, 19.94077354, 15.96653664,  2.06504746]), array([228.08601921,   0.88175645, 226.49442573,   3.2922475 ]), array([24.9744608 , 10.66526163, 25.36304192, 28.31991828]), array([202.29418941,   4.14204471, 171.18429003,  12.34052383]), array([148.48083364,   0.8920642 , 219.96882803,   0.        ]), array([155.43749847,   2.87161574, 144.7427331 ,   7.79450478]), array([28.55331011,  2.06355491, 54.66207321,  0.        ]), array([152.64198746,  12.54009534, 156.98066494,  14.72749899]), array([166.48563569,   3.33625212, 171.18322194,   1.03035513]), array([132.10112352,   0.        ,  47.01487118,   0.47975884]), array([169.77187239,   7.33604791, 126.68956328,   7.75064625]), array([210.1281585 ,   0.        , 207.07486372,   1.88458625]), array([ 1.89999062, 14.01429056,  2.30020931, 15.85319424]), array([11.49551264,  6.

Episode: 33/800, Epsilon:0.95, Average reward: -244.11
Prediction for [500,0,500,0] is: [[-480.3235  -524.38074 -261.31528 -299.87964 -557.18146]]
[array([10.15537272,  6.35569576,  8.58159962,  4.80307828]), array([239.05290739,   0.        , 235.45820234,   0.        ]), array([107.9503405 ,   2.34977794, 148.55301426,   2.16344722]), array([233.30091879,   1.91150684, 236.04910569,   1.14556521]), array([150.0179471 ,  25.31689043,  57.41554292,   2.79304227]), array([68.83095151,  6.3367992 , 22.66065303,  2.39193083]), array([163.33221558,   0.        , 163.43630983,   8.30188193]), array([216.18634585,   0.        , 240.59560211,   0.46480478]), array([131.2929676 ,   0.        ,  96.80388743,   1.06920638]), array([124.93232327,   0.        , 177.75333629,   1.82780225]), array([ 1.2970537 ,  4.22057353, 14.75395552, 14.6969582 ]), array([ 77.62485616,   0.        , 160.76985579,   0.        ]), array([ 26.43129149,   0.        , 164.07262616,   3.80061557]), array([182.11201412

In [ ]:
Demo_Mode = True
if __name__ == "__main__":
    # Initialize storage
    reward_storage = []
    
    # If previous agents aren't to be loaded, proceed to memory population
    if not load_trained:
        # Initialize simulation
        if 'Vissim' not in globals() or Vissim == None:
            Vissim, Simulation, Network, cache_flag = COMServerDispatch(model_name, vissim_working_directory,\
                                                                        memory_population_length, Start_Fresh, reset_flag = True)
        else:
            Vissim = com.Dispatch("Vissim.Vissim")
            Simulation, Network = COMServerReload(Vissim, model_name, vissim_working_directory,\
                                                  memory_population_length, Start_Fresh, reset_flag = True)
        
    # Setting Random Seed
    Vissim.Simulation.SetAttValue('RandSeed', Random_Seed)
    print ('Random seed set in simulator. Random Seed = '+str(Random_Seed))

    # Deploy Network Parser (crawl network)
    npa = NetworkParser(Vissim)
    print('NetworkParser has succesfully crawled the model network.')
    
    # Initialize agents
    if program == "DQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, Vissim, DoubleDQN = False, Dueling = False) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, Vissim, DoubleDQN = True, Dueling = False) for ID in npa.signal_controllers_ids] 
        agents_deployed = True
    elif program == "DuelingDQN":
        Agents = [DQNAgent(state_size, action_size, ID, state_type, npa, memory_size, gamma, 0 if Demo_Mode else epsilon_start,\
                           epsilon_end, epsilon_decay, alpha, copy_weights_frequency, Vissim, DoubleDQN = True, Dueling = True) for ID in npa.signal_controllers_ids] 
      
        agents_deployed = True
    else:
        print("Incorrect Agent Class selected. Deployment could not be completed.")
        quit()
    
    if agents_deployed:
        print("Deployed {} agent(s) of the Class {}.".format(len(Agents), program))
    
    if Demo_Mode:
        for index, agent in enumerate(Agents):
            Filename = os.path.join(vissim_working_directory, model_name, model_name+'_'+ Session_ID + '_Agent'+str(index)+'.h5')
            agent.model = load_model(Filename)
            Memory_Filename = os.path.join(vissim_working_directory, model_name, model_name+'_'+ Session_ID + '_Agent'+str(index)+'_Memory'+'.p')
            agent.memory = pickle.load(open(Memory_Filename, 'rb'))
        print('Items successfully loaded.')        
        SF.run_simulation_episode(Agents, Vissim, state_type, state_size, memory_population_length, Demo_Mode = True)
        Vissim = None

In [ ]:
 # Plotting training progress
    plt.figure(figsize=(8,4.5))
    x_series = range(1,len(reward_storage)+1)
    fit = np.polyfit(x_series,reward_storage,1)
    fit_fn = np.poly1d(fit) 
    plt.plot(x_series,reward_storage, '-b', x_series, fit_fn(x_series), '--r')
    plt.xlabel('Episodes')
    plt.ylabel('Average ageng reward in episode')
    plt.title('Training evolution and trend')
    plt.gca().legend(('Episode Reward','Linear Trend'))
    plt.show()
    

In [ ]:
# Plotting training loss
bo = loss/np.max(loss)
x = np.arange(0, len(bo), 128)
plt.figure(figsize=(16,9))
x_series = range(1,len(bo)+1)
plt.plot(x_series,np.sqrt(bo), '-b')
#for xc in x:
#    plt.axvline(xc)
plt.xlabel('Training Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.gca().legend(('Loss'))
plt.show()

In [ ]:
Agents[0].model.predict(np.reshape([0,5,0,5],(1,4)))[0]

In [ ]:
minibatch = random.sample(Agents[0].memory,2)
minibatch[0][0][0]

In [ ]:
sta = [(5 , 0 ,5, 0),(6 , 0 ,6, 0)]
st= np.asarray(sta)
print(st)
tar = [(5,0,5,0,5),(6,0,6,0,6)]
tg = np.asarray(tar)
Agents[0].model.fit(st,tg,epochs=1, verbose=0)

In [ ]:
st = np.asarray(state)
tg = np.asarray(target)
b.model.fit(st,tg,epochs=1,verbose=0)

In [ ]:
epsilon_decay = 1.2(epsilon_end - epsilon_start)/(episodes-1)
epsilon_decay

In [ ]:
np.zeros(batch_size, 4)